___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

Of course we can perform more advanced operations than simply buying and selling on specific time events
As an example, also:

1. Monthly buying - Cost Averaging Effect
2. Buy/Sell one equity based on the price of another equity
3. Buy/Sell based on time of year (seasonal)

# Scheduling Functins

## Monthly buying - Cost Averaging Effect

Another possibility, instead of the initial, full investment is to buy on a regular schedule (e.g monthly).
The idea behind this tactic is that we buy during highs and during lows to create a cost average effect

To do so we can make use of the schedule functionality provided by quantconnect: **self.Schedule.On(DateRules ,TimeRules, function)** <br />
(https://www.quantconnect.com/docs/algorithm-reference/scheduled-events)
**DateRules** define on what date to buy and the TimeRules define what time to buy on the given date.
The third argument is the function which is called each time the schedule is triggered

To define our monthly buy function we create a scheduled event, where the DateRules is MonthStart() and the TimeRules is 9:30am. Thus the event will be executed at 930 on the first trading day of each month.
If executed we cann the self.Buy function which is defined below: <br />
Note that we don't use **SetHoldings()** to invest but **MarketOrder(stock, num_shares)** which creates a market order to buy num_shares stocks of the given symbol.

```python
def Initialize(self):

    self.SetCash(10000)
    self.SetStartDate(2010,1,1) 
    self.apple = self.AddEquity("AAPL", Resolution.Daily)
    
    self.Schedule.On(self.DateRules.MonthStart(), self.TimeRules.At(9,30), self.Buy)
    self.monthly_buy = 200

```

```python
def Buy(self):
    """
    Buy function which is called on the first trading day of each month
    """

    # If we run out of money, dont buy anymore! End of investing
    if self.Portfolio.Cash < self.apple.Open:
        self.Log("Not enough money! Stop investing")
        return

    # Calculate how many shares we can buy with 200$
    shares_to_buy = int(self.monthly_buy / self.apple.Open)
    self.Log(f"Buying {shares_to_buy} of {self.apple.Symbol}. Remaining money: {self.Portfolio.Cash} $")
    self.MarketOrder("AAPL", shares_to_buy)
```

As we create the order in the **buy** function, we can skip defining the OnData method. <br />
Let's say we want to invest 200$ into Apple each month until we run out of money.

In [ ]:
class MonthlyBuy(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(10000)
        self.SetStartDate(2010,1,1) 
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
        self.Schedule.On(self.DateRules.MonthStart(), self.TimeRules.At(9,30), self.Buy)
        self.monthly_buy = 200

    def Buy(self):
        """
        Buy function which is called on the first trading day of each month
        """

        # If we run out of money, dont buy anymore! End of investing
        if self.Portfolio.Cash < self.apple.Open:
            self.Log("Not enough money! Stop investing")
            return
        
        # Calculate how many shares we can buy with 200$
        shares_to_buy = int(self.monthly_buy / self.apple.Open)
        self.Log(f"Buying {shares_to_buy} of {self.apple.Symbol}. Remaining money: {self.Portfolio.Cash} $")
        self.MarketOrder("AAPL", shares_to_buy)

The problem with above code is that, given the price of an apple share rises above 200\\$ we would never buy again.
To solve this, we can use a "cash reserve" variable that receives an additional amount of 200\\$ each month if we were not able to buy a stock.

We will use tesla as an example for this:

In [ ]:
class MonthlyBuy2(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(10000)
        self.SetStartDate(2019,1,1) 
        self.tesla = self.AddEquity("TSLA", Resolution.Daily)
        self.Schedule.On(self.DateRules.MonthStart(), self.TimeRules.At(9,30), self.Buy)
        self.monthly_buy = 200
        self.cash_reserve = 200  # Used to store our money

    def Buy(self):
        """
        Buy function which is called on the first trading day of each month
        """

        # If we run out of money, dont buy anymore! End of investing
        if self.Portfolio.Cash < self.tesla.Open:
            self.Debug("Not enough money! Stop investing")
            return
        
        # If the stock price is larger than our cash_reserve,
        # we increase our cash_reserve and dont buy anything this month
        elif self.tesla.Open > self.cash_reserve:
            self.cash_reserve += self.monthly_buy

            self.Log(f"Stock is too expensive ({self.tesla.Open} $), add {self.monthly_buy} $ to cash_reserve. \
                         Current amount: {self.cash_reserve} $")
            return
        
        # Calculate how many shares we can buy with the amount in our cash_reserve
        shares_to_buy = int(self.cash_reserve / self.tesla.Open)
        self.Log(f"Buying {shares_to_buy} of {self.tesla.Symbol}. Remaining money: {self.Portfolio.Cash} $")
        self.MarketOrder(self.tesla.Symbol, shares_to_buy)
        # Reset cash_reserve
        self.cash_reserve = self.monthly_buy

## Buy a stock based on the price of another stock.
As an example: Let's invest in VNQ (Vanguard Real Estate Index Fund) and VNQI (Vanguard Real Estate Index Fund International). <br />
Depending on the Open Close price difference of the day we either invest in VNQ or VNQI

In [ ]:
class PriceComparison(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2018, 1, 1)  # We want to invest on the first January 2010
        self.SetEndDate(2020, 1, 1)
        self.SetCash(10000)  #  10,000$ are available
        
        self.vnq = self.AddEquity("VNQ", Resolution.Daily)
        self.vnqi = self.AddEquity("VNQI", Resolution.Daily)

    def OnData(self, data):
        
        # Compute percent price difference between VNQI and VNQ
        vnqi_pop = (self.vnqi.Close - self.vnqi.Open)/self.vnqi.Open
        vnq_pop = (self.vnq.Close - self.vnq.Open)/self.vnq.Open
        
        # IF DAILY OPEN PRICE OF VNQI is 2% greater than VNQ, set holdings to VNQ
        if  vnqi_pop > 0.02 and vnqi_pop>vnq_pop:   
            self.SetHoldings("VNQ", 1,True)
            self.Log(f'VNQI POP: {vnqi_pop}')
        
        # IF DAILY OPEN PRICE OF VNQI is 2% less than VNQ, set holdings to VNQI
        if  vnq_pop > 0.02 and vnq_pop>vnqi_pop:   
            self.SetHoldings("VNQI", 1,True)
            self.Log(f'VNQ POP: {vnq_pop}')
            
        else:
            self.Log('No Action Taken Today')
             
            return
        

